In [2]:
import pyodbc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import xml.etree.ElementTree as ET
import pandas as pd
import os
import re

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing

from sklearn.feature_selection import SelectFromModel, SequentialFeatureSelector
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from scipy import stats
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
 
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import r2_score
from statsmodels.api import OLS, add_constant

<p style="font-size: 16px; color: blue;"> 可用函式 </p>

In [3]:
def strip_all_columns(df):
    df_cleaned = df.copy()
    for col in df_cleaned.columns:
        if pd.api.types.is_string_dtype(df_cleaned[col]) or df_cleaned[col].dtype == object:
            df_cleaned[col] = df_cleaned[col].astype(str).str.strip()
    return df_cleaned

In [4]:
def count_na_and_blank(df):
    result = {}
    for col in df.columns:
        na_count = df[col].isna().sum()
        blank_count = 0
        if df[col].dtype == object or pd.api.types.is_string_dtype(df[col]):
            blank_count = df[col].astype(str).apply(lambda x: x.strip() == '').sum()
        result[col] = {'na': na_count, 'blank_or_space': blank_count}
    return pd.DataFrame(result).T

# summary = count_na_and_blank(df_merge) 
# summary = summary[(summary['na'] > 0) | (summary['blank_or_space'] > 0)]
# print(summary)

In [ ]:
# zms2: 分出L1/2 -> L1 L2
def split_layer_rows(df):
    df_base = df.copy()

    mask = (
        (df_base['mtl_type'] == '基板') &
        (df_base['layer_2'].str.contains('/')) &
        (~df_base['layer_2'].str.contains('-')) &
        (~df_base['layer_2'].str.contains('L0'))
    )

    # 不需拆分的資料保留
    df_keep = df_base[~mask].copy()

    # 要拆分的資料
    df_split = df_base[mask].copy()

    # 新資料列清單
    expanded_rows = []

    for _, row in df_split.iterrows():
        try:
            layer_range = row['layer_2'].replace('L', '')  # 去除 'L'
            start, end = map(int, layer_range.split('/'))  # 拆成起始與結束層數
            for i in range(start, end + 1):
                new_row = row.copy()
                new_row['layer_2'] = f'L{i}'
                expanded_rows.append(new_row)
        except:
            continue  # 若轉換失敗則略過

    # 合併原本資料 + 展開資料
    df_result = pd.concat([df_keep, pd.DataFrame(expanded_rows)], ignore_index=True)

    # 可選擇排序（若有需要）
    # df_result = df_result.sort_values(by=['prod_nbr', 'item_nbr', 'layer_2'], ignore_index=True)

    return df_result


<p style="font-size: 16px; color: blue;"> SQL Server 連接配置 </p>

In [5]:
# # DM
server = 'NTEMES\MES'
username = 'CHIAFU\wilson_liu'
database = 'IM'
trusted_conn = 'yes'

connection_string = f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};Trusted_Connection={trusted_conn};'

conn = pyodbc.connect(connection_string)

# 殘銅
server = 'nte403'
username = 'CHIAFU\wilson_liu'
database = 'chiafu'
trusted_conn = 'yes'

connection2_string = f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};Trusted_Connection={trusted_conn};'

conn2 = pyodbc.connect(connection2_string)

# 壓合程式
server = 'nte403'
username = 'CHIAFU\wilson_liu'
database = 'wwip'
trusted_conn = 'yes'

connection5_string = f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};Trusted_Connection={trusted_conn};'

conn5 = pyodbc.connect(connection5_string)

# recipe
server = 'nte403'
username = 'CHIAFU\wilson_liu'
database = 'wwip'
trusted_conn = 'yes'

connection4_string = f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};Trusted_Connection={trusted_conn};'

conn4 = pyodbc.connect(connection4_string)

# bsitemm
server = 'nte403'
username = 'CHIAFU\wilson_liu'
database = 'erp'
trusted_conn = 'yes'

connection3_string = f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};Trusted_Connection={trusted_conn};'

conn3 = pyodbc.connect(connection3_string)

<p style="font-size: 16px; color: blue;"> DM </p>

In [6]:
# 查詢資料 JOIN (用AVGX 和 AVGY)
query_join_IM = '''
SELECT 
    m.idx, 
    m.Part_nbr, 
    m.Lot_nbr, 
    m.Layer, 
    m.CamX, 
    m.CamY,
    d.Layer AS d_Layer,
    d.PSVariationX AS d_PSVarX, 
    d.PSVariationY AS d_PSVarY,
    d.AVGX,
    d.AVGY,
    s.kind
FROM 
    DRY_XrayProduction_M m
    
LEFT JOIN 
    DRY_XrayProduction_D d ON m.idx = d.idx

LEFT JOIN 
    nte403.chiafu.dbo.slss s ON m.Part_nbr = s.part_nbr
WHERE 
    m.Credate > '2024-01-01' 
    AND NOT (m.Credate >= '2025-01-01' AND m.Credate <= '2025-05-26')
'''
# m.Layer = d.Layer 不會對應到 811S002A4GN

In [7]:
df_merge = pd.read_sql(query_join_IM, conn)
df_merge = strip_all_columns(df_merge)
df_merge = df_merge.drop(columns=['idx'])
# df_merge = df_merge[df_merge['kind'] != 'N']
df_merge = df_merge[df_merge['kind'] == 'I']
df_merge = df_merge.drop(columns=['kind'])
df_merge = df_merge.drop_duplicates()


C:\Users\wilson_liu\AppData\Local\Temp\ipykernel_21288\1743552340.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_merge = pd.read_sql(query_join_IM, conn)


<p style="font-size: 16px; color: blue;">殘銅</p>

In [8]:
# 查詢資料 JOIN 殘銅
# 串入M 檔的料號，先串好，再挑時間

query_join_zms = '''
SELECT
    a1.prod_nbr, a1.layer AS layer_full, a1.iss_x, a1.iss_y, a1.creat_date,
    s6.it3, s6.it5, s6.layer
FROM 
    zms1a a1
INNER JOIN 
    zms6 s6
    ON a1.prod_nbr = s6.prod_nbr AND a1.layer = s6.layer    
WHERE 
    s6.cused='51' 
    AND a1.creat_date >= '2024-01-01' AND NOT ( a1.creat_date >= '2025-01-01' AND a1.creat_date <= '2025-05-26') 
'''

df_zms = pd.read_sql(query_join_zms, conn2)
df_zms =df_zms.sort_values(by='creat_date', ascending=False)
df_zms = strip_all_columns(df_zms)

C:\Users\wilson_liu\AppData\Local\Temp\ipykernel_21288\669108922.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_zms = pd.read_sql(query_join_zms, conn2)


In [9]:
# 轉回數字格式
df_zms['it3'] = pd.to_numeric(df_zms['it3'], errors='coerce')
df_zms['it5'] = pd.to_numeric(df_zms['it5'], errors='coerce')
df_zms['iss_x'] = pd.to_numeric(df_zms['iss_x'], errors='coerce')
df_zms['iss_y'] = pd.to_numeric(df_zms['iss_y'], errors='coerce')

#  C面 / S面 殘銅率
denominator = (df_zms['iss_x'] * df_zms['iss_y'] / 92903.04) + 0.001
df_zms['C-Residual Copper Rate'] = df_zms['it3'] / denominator
df_zms['S-Residual Copper Rate'] = df_zms['it5'] / denominator

df_zms = df_zms[['prod_nbr', 'layer', 'C-Residual Copper Rate', 'S-Residual Copper Rate']]
df_zms = df_zms.drop_duplicates()

df_zms.shape

(13531, 4)

In [10]:
# 複製一份 Detail，避免原表被修改
df_detail_copy = df_merge.copy()

# Step 1: 展開 df_residual 表的 L6/7 → L6 和 L7 各一列
df_residual_expanded = []

for _, row in df_zms.iterrows():
    layer_raw = row['layer'].replace(" ", "")
    raw_parts = layer_raw.split("/")  # e.g. ['L6', '7']
    
    prod_nbr = row['prod_nbr']
    c_rate = row['C-Residual Copper Rate']
    s_rate = row['S-Residual Copper Rate']

    # 強制每個片段都轉成 L+數字（避免只有數字或奇怪格式）
    def normalize(layer_str):
        m = re.search(r'\d+', layer_str)
        return f'L{m.group()}' if m else layer_str

    if len(raw_parts) == 2:
        layer1 = normalize(raw_parts[0])
        layer2 = normalize(raw_parts[1])

        df_residual_expanded.append({
            'prod_nbr': prod_nbr,
            'd_Layer': layer1,
            'Residual Copper Rate': c_rate
        })
        df_residual_expanded.append({
            'prod_nbr': prod_nbr,
            'd_Layer': layer2,
            'Residual Copper Rate': s_rate
        })


# 建立新的展平後的殘銅率表
df_residual_flat = pd.DataFrame(df_residual_expanded)
df_residual_flat = df_residual_flat.drop_duplicates()

In [11]:
# # Step 2: 合併：以 Part_nbr = prod_nbr, d_Layer = d_Layer 對應
# df_merged_2 = pd.merge(
#     df_detail_copy,
#     df_residual_flat,
#     left_on=['Part_nbr', 'd_Layer'],
#     right_on=['prod_nbr', 'd_Layer'],
#     how='left'
# )

# # 如果不需要 prod_nbr 欄位可以刪掉
# df_merged_2.drop(columns=['prod_nbr'], inplace=True)

# # 結果在 df_merged 中，多了一欄 'Residual Copper Rate'
# df_merged_2 = df_merged_2.drop_duplicates()
# # df_merged_2

<p style="font-size: 16px; color: blue;">壓合程式、壓合recipe</p>

In [12]:
# df_cpinm( part_nbr、lot_nbr、layer) and credate -> df_merge_IM_zms ('Part_nbr', 'Lot_nbr', 'Layer')
# df_cpinm(壓合recipe)-> RMS_PinlamRecipe (RecipeName)

query_cpinm = "SELECT macno as 'MacNo',in_code as 'MacRecipe' , * FROM cpinm WHERE credate >= '2024-01-01' AND NOT (credate >= '2025-01-01' AND credate <= '2025-05-26') "
df_cpinm = pd.read_sql(query_cpinm, conn5)
df_cpinm = strip_all_columns(df_cpinm)

C:\Users\wilson_liu\AppData\Local\Temp\ipykernel_21288\1781157386.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_cpinm = pd.read_sql(query_cpinm, conn5)


In [13]:
df_cpinm = df_cpinm[['MacNo', 'MacRecipe', 'lot_nbr', 'layer', 'part_nbr']]
df_cpinm = df_cpinm.drop_duplicates()

In [14]:
# # 合併 prod_nbr 和 Part_nbr
# df_merged_3 = pd.merge(df_merged_2, df_cpinm, left_on=['Lot_nbr', 'Part_nbr', 'Layer'],  right_on=['lot_nbr', 'part_nbr', 'layer'], how='inner')
# df_merged_3 = df_merged_3.drop(columns=['lot_nbr','layer','part_nbr'])
# df_merged_3 = df_merged_3.drop_duplicates()

<p style="font-size: 16px; color: blue;">RC、廠商和基板型號</p>

In [182]:
# 以下三個串 RC

query_zms2 = "SELECT * FROM zms2"
df_zms2 = pd.read_sql(query_zms2, conn2)
df_zms2 = strip_all_columns(df_zms2)

query_zbom2 = "SELECT * FROM zbom2"
df_zbom2 = pd.read_sql(query_zbom2, conn2)
df_zbom2 = strip_all_columns(df_zbom2)

query_bsitemm = "SELECT * FROM bsitemm"
df_bsitemm = pd.read_sql(query_bsitemm, conn3)
df_bsitemm = strip_all_columns(df_bsitemm)

C:\Users\wilson_liu\AppData\Local\Temp\ipykernel_21288\2876753893.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_zms2 = pd.read_sql(query_zms2, conn2)
C:\Users\wilson_liu\AppData\Local\Temp\ipykernel_21288\2876753893.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_zbom2 = pd.read_sql(query_zbom2, conn2)
C:\Users\wilson_liu\AppData\Local\Temp\ipykernel_21288\2876753893.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_bsitemm = pd.read_sql(query_bsitemm, conn3)


In [183]:
# 用df_merge篩選df_zms2，為 df_zms2_sub

# 1. 把 df_merge 的key value組合取出來，轉成唯一列表
key_pairs = (
    df_merge[['Part_nbr', 'Layer']]
    .drop_duplicates()
    .rename(columns={'Part_nbr': 'prod_nbr', 'Layer': 'layer'})   # 改名方便後面 inner merge
)

# 2. 直接 inner merge，保留「兩邊共有」的 (prod_nbr, layer)
df_zms2_sub = (
    df_zms2
      .merge(key_pairs, on=['prod_nbr', 'layer'], how='inner')
      .reset_index(drop=True)
)

In [184]:
x = split_layer_rows(df_zms2_sub)

# 去掉 mtl_desc3、grade，都是' '
x = x.drop(columns=['mtl_desc3', 'grade', 'mtl_flag', 'use_qty', 'mtl_desc4', 'mtl_desc5', 'layer_2ThickTole'])

In [185]:
x = x.sort_values(by=['prod_nbr', 'layer', 'item_nbr', 'layer_2']).reset_index(drop=True)

In [186]:
x['prod_nbr'].unique()

array(['107S002C48N', '107S003C3BB', '107S004C78C', ..., '999S790AB8G',
       '999S798A78N', '999S800A03B'], shape=(1835,), dtype=object)

<p style="font-size: 14px; color: green;"> df_zms2_sub 整理成 1 to 1</p>

<p style="font-size: 12px; color: red;"> check料號: 120SO07DA8C、999S790AB8G</p>

In [295]:
# x[x['prod_nbr']=='999S790AB8G']

In [288]:
# 預處理：保留原始索引並排序
df = x.sort_values(by=['prod_nbr', 'layer', 'item_nbr']).reset_index()
df['item_nbr'] = pd.to_numeric(df['item_nbr'], errors='coerce')

# 初始化所有 plate_ 欄位
cols_to_plate = [col for col in df.columns 
                if col not in ['prod_nbr', 'layer', 'item_nbr', 'mtl_type', 'index']]
for col in cols_to_plate:
    df[f'plate_{col}'] = None

drop_indexes = []
result_df = df.copy()

# 處理每個 group
for (prod, layer), group in df.groupby(['prod_nbr', 'layer']):
    group = group.sort_values('item_nbr').reset_index(drop=True)
    
    # 1. 先處理基板之間的膠片（嚴格按照基板-膠片-膠片-基板模式）
    substrate_indices = group[group['mtl_type'] == '基板'].index.tolist()
    
    for i in range(len(substrate_indices)-1):
        start_idx = substrate_indices[i]
        end_idx = substrate_indices[i+1]
        
        # 檢查是否為連續相同item_nbr的基板
        if (group.loc[start_idx, 'item_nbr'] == group.loc[end_idx, 'item_nbr'] and
            group.loc[start_idx, 'mtl_type'] == '基板' and
            group.loc[end_idx, 'mtl_type'] == '基板'):
            
            adhesives = group.loc[start_idx+1:end_idx-1]
            adhesives = adhesives[adhesives['mtl_type'] == '膠片']
            
            if len(adhesives) >= 1:
                # 第一個膠片給第一個基板
                for col in cols_to_plate:
                    result_df.loc[group.loc[start_idx, 'index'], f'plate_{col}'] = adhesives.iloc[0][col]
            
            if len(adhesives) >= 2:
                # 最後一個膠片給第二個基板
                for col in cols_to_plate:
                    result_df.loc[group.loc[end_idx, 'index'], f'plate_{col}'] = adhesives.iloc[-1][col]
            
            drop_indexes.extend(adhesives['index'].tolist())
    
    # 2. 處理銅箔（成對處理）
    copper_indices = group[group['mtl_type'] == '銅箔'].index.tolist()
    
    # 確保銅箔是成對出現
    for i in range(0, len(copper_indices), 2):
        if i+1 >= len(copper_indices):
            break  # 忽略最後一個單獨的銅箔
            
        start_idx = copper_indices[i]
        end_idx = copper_indices[i+1]
        
        # 取得銅箔之間的膠片
        adhesives = group.loc[start_idx+1:end_idx-1]
        adhesives = adhesives[adhesives['mtl_type'] == '膠片'].sort_values('item_nbr')
        
        if len(adhesives) == 0:
            continue
        
        # 判斷分配方式（基於距離）
        first_dist_to_prev = abs(adhesives.iloc[0]['item_nbr'] - group.loc[start_idx, 'item_nbr'])
        first_dist_to_next = abs(adhesives.iloc[0]['item_nbr'] - group.loc[end_idx, 'item_nbr'])
        
        if first_dist_to_prev <= first_dist_to_next:
            # 第一個膠片給前銅箔，最後一個給後銅箔
            for col in cols_to_plate:
                result_df.loc[group.loc[start_idx, 'index'], f'plate_{col}'] = adhesives.iloc[0][col]
                if len(adhesives) >= 2:
                    result_df.loc[group.loc[end_idx, 'index'], f'plate_{col}'] = adhesives.iloc[-1][col]
        else:
            # 第一個膠片給後銅箔，最後一個給前銅箔
            for col in cols_to_plate:
                result_df.loc[group.loc[end_idx, 'index'], f'plate_{col}'] = adhesives.iloc[0][col]
                if len(adhesives) >= 2:
                    result_df.loc[group.loc[start_idx, 'index'], f'plate_{col}'] = adhesives.iloc[-1][col]
        
        # 標記所有膠片要刪除
        drop_indexes.extend(adhesives['index'].tolist())
    
    # 3. 處理其他非膠片之間的膠片（一般情況）
    non_adhesive_indices = group[group['mtl_type'].isin(['基板', '銅箔'])].index.tolist()
    
    for i in range(len(non_adhesive_indices)-1):
        start_idx = non_adhesive_indices[i]
        end_idx = non_adhesive_indices[i+1]
        
        # 跳過已處理的基板和銅箔
        if (group.loc[start_idx, 'mtl_type'] == '基板' and 
            group.loc[end_idx, 'mtl_type'] == '基板' and
            group.loc[start_idx, 'item_nbr'] == group.loc[end_idx, 'item_nbr']):
            continue
        
        if (group.loc[start_idx, 'mtl_type'] == '銅箔' and 
            group.loc[end_idx, 'mtl_type'] == '銅箔'):
            continue
        
        # 一般情況處理
        adhesives = group.loc[start_idx+1:end_idx-1]
        adhesives = adhesives[adhesives['mtl_type'] == '膠片'].sort_values('item_nbr')
        
        if len(adhesives) == 0:
            continue
        
        if len(adhesives) == 1:
            first_adhesive = adhesives.iloc[0]
            second_adhesive = first_adhesive.copy()
        else:
            first_adhesive = adhesives.iloc[0]
            second_adhesive = adhesives.iloc[-1]
            for j in range(1, len(adhesives)-1):
                drop_indexes.append(adhesives.iloc[j]['index'])
        
        # 分配膠片
        for col in cols_to_plate:
            result_df.loc[group.loc[start_idx, 'index'], f'plate_{col}'] = first_adhesive[col]
            result_df.loc[group.loc[end_idx, 'index'], f'plate_{col}'] = second_adhesive[col]
        
        drop_indexes.extend([first_adhesive['index'], second_adhesive['index']])

# 刪除所有膠片
x2 = result_df.drop(index=drop_indexes).reset_index(drop=True)
x2 = x2[x2['mtl_type'] != '膠片']

# 可選：刪除完全為空的 plate_ 欄位
x2 = x2.dropna(axis=1, how='all')

In [289]:
x2 = x2.drop(columns=['plate_layer_2'])

In [293]:
# x2[x2['prod_nbr']=='120SO07DA8C'].sort_values(by='item_nbr',ascending=True)

In [306]:
a = x2[x2['prod_nbr']=='999S790AB8G'].sort_values(by=['item_nbr', 'layer_2'],ascending=True)
a = a.reset_index(drop=True)

In [307]:
a = a.drop(columns=['index'])



,prod_nbr,layer,item_nbr,mtl_type,mtl_desc1,mtl_desc2,vend_nbr,layer_2,layer_2Thick,plate_mtl_desc1,plate_mtl_desc2,plate_vend_nbr,plate_layer_2Thick
0,999S790AB8G,L1/24,1,銅箔,0.5oz(Pin Lam),0.015,李長榮,L1,1.9,1078LR,0.079,台光,2.66
1,999S790AB8G,L1/24,3,基板,0.102mm_1/1-RG312+1086*1,0.173,台光,L2,6.3,1078LR,0.079,台光,2.66
2,999S790AB8G,L1/24,3,基板,0.102mm_1/1-RG312+1086*1,0.173,台光,L3,6.3,1035SR,0.152,台光,
3,999S790AB8G,L1/24,6,基板,0.102mm_1/1-RG312+1086*1,0.173,台光,L4,6.3,1078LR,0.0,台光,5.16
4,999S790AB8G,L1/24,6,基板,0.102mm_1/1-RG312+1086*1,0.173,台光,L5,6.3,1035SR,0.152,台光,
5,999S790AB8G,L1/24,9,基板,0.127mm_1/2-RG312/RTF+1078*2,0.234,台光,L6,8.5,1078LR,0.0,台光,5.16
6,999S790AB8G,L1/24,9,基板,0.127mm_1/2-RG312/RTF+1078*2,0.234,台光,L7,8.5,1078HR,0.315,台光,
7,999S790AB8G,L1/24,13,基板,0.11mm_2/2-RTF+1035*2,0.244,台光,L8,8.56,1078HR,0.0,台光,10.96
8,999S790AB8G,L1/24,13,基板,0.11mm_2/2-RTF+1035*2,0.244,台光,L9,8.56,1078HR,0.315,台光,
9,999S790AB8G,L1/24,17,基板,0.127mm_2/1-RG312/RTF+1078*2,0.234,台光,L10,8.5,1078HR,0.0,台光,10.96


In [308]:
# a.to_excel('a.xlsx', index=False)

<p style="font-size: 14px; color: green;"> RC value</p>

In [297]:
# 與RC SQL code相應
# 選出 ver 最大
df_zbom2['ver'] = pd.to_numeric(df_zbom2['ver'], errors='coerce') # ver 字串變數字，不然'10' < '3'
df_zbom2['rk'] = df_zbom2.groupby(['prod_nbr', 'mtl_type'])['ver'].rank(method='first', ascending=False)
step0 = df_zbom2[df_zbom2['rk'] == 1].copy() # step0 是最新版的BOM

# 兩次條件 join，然後 coalesce 結果
#  先對"非基板"條件
merge1 = df_zms2_sub.merge(
    step0,
    left_on=['prod_nbr', 'layer', 'mtl_type', 'item_nbr'],
    right_on=['prod_nbr', 'layer', 'mtl_type', 'mtl_types'],
    how='left'
)

# 再對"基板"條件（layer_2）
merge2 = df_zms2_sub.merge(
    step0,
    left_on=['prod_nbr', 'layer_2', 'mtl_type', 'item_nbr'],
    right_on=['prod_nbr', 'layer', 'mtl_type', 'mtl_types'],
    how='left'
)

df_merge_2 = df_zms2_sub.copy()
df_merge_2['itemno_1_b1'] = merge1['itemno_1'] # 兩次merge
df_merge_2['itemno_1_b2'] = merge2['itemno_1']
df_merge_2['itemno_1'] = df_merge_2['itemno_1_b1'].combine_first(df_merge_2['itemno_1_b2']) # 如果itemno_1_b1有值，就用它，不然用itemno_1_b2

df_final = df_merge_2.merge(
    df_bsitemm[['itemno', 'itemcnm']],
    left_on='itemno_1',
    right_on='itemno',
    how='left'
)

In [298]:
df_final

,prod_nbr,layer,item_nbr,mtl_type,mtl_desc1,mtl_desc2,mtl_desc3,grade,mtl_flag,vend_nbr,...,layer_2,mtl_desc4,mtl_desc5,layer_2Thick,layer_2ThickTole,itemno_1_b1,itemno_1_b2,itemno_1,itemno,itemcnm
0,120SQ83AA8C,L1/22,04,膠片,1078MR,0.168,,,N,台光,...,,EM-89B,TG-175 無鹵素,,,NaN,NaN,NaN,NaN,NaN
1,120SQ83AA8C,L1/22,21,基板,0.08mm_H/1-HVLP+1078*1,0.130,,,N,台光,...,L14/15,EM-890,TG-175 無鹵素,4.7,"+0.5mil,-0.5mil",NaN,NaN,NaN,NaN,NaN
2,120SF57D48F,L1/8,05,膠片,2116HR,0.000,,,N,南亞,...,,NPG-170DB,TG-170 無鹵素,None,None,NaN,NaN,NaN,NaN,NaN
3,120SF57D48F,L1/8,06,膠片,2116HR,0.000,,,N,南亞,...,,NPG-170DB,TG-170 無鹵素,None,None,NaN,NaN,NaN,NaN,NaN
4,120SF57D48F,L1/8,07,基板,0.13mm_1/1,0.198,,,N,南亞,...,L4/5,NPG-170D,TG-170 無鹵素,None,None,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35499,187S113CA8C,L1/22,30,基板,0.102mm_H/H-HVLP2+1035*2,0.137,,,N,台光,...,L20/21,EM-890K,TG-170 無鹵素,5.1,"+0.5mil,-0.5mil",NaN,NaN,NaN,NaN,NaN
35500,187S113CA8C,L1/22,27,基板,0.102mm_H/H-HVLP2+1035*2,0.137,,,N,台光,...,L18/19,EM-890K,TG-170 無鹵素,5.1,"+0.5mil,-0.5mil",NaN,NaN,NaN,NaN,NaN
35501,187S113CA8C,L1/22,17,膠片,1078LR,0.000,,,N,台光,...,,EM-89BK,TG-170 無鹵素,5,"+0.7mil,-0.7mil",NaN,NaN,NaN,NaN,NaN
35502,120SP98A5BC,L1/10,15,膠片,1080,0.079,,,N,南亞,...,,NPG-170DB,TG-170 無鹵素,2.86,"+0.5mil,-0.5mil",NaN,NaN,NaN,NaN,NaN


In [296]:
df_zbom2

,prod_nbr,layer,mtl_type,mtl_types,itemno_1,itemno_2,itemno_3,ver
0,000-000A21,L1/4,基板,03,,,,
1,000-000A21,L1/4,銅箔,01,C011,,,
2,000-000A21,L1/4,銅箔,05,C011,,,
3,000-000A21,L1/4,膠片,02,B111,B115,,
4,000-000A21,L1/4,膠片,04,B115,B111,,
...,...,...,...,...,...,...,...,...
818034,X200A23,L1/4,銅箔,05,C011,,,
818035,X200A23,L1/4,膠片,02,B115,,,
818036,X200A23,L1/4,膠片,04,B115,,,
818037,X200A23,L2/3,,01,,,,
